In [3]:
import backtrader as bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

# 数据准备

In [2]:
from update_sql import update_sql
#更新数据库
update_sql(table_name='daily_data')

数据已经是最新的！


In [48]:
from base import sql_engine,ts_pro
pro=ts_pro()
engine = sql_engine()
def get_data(code):
    sql=f"select * from daily_data where trade_date>'20150201' and ts_code='{code}'"
    data=pd.read_sql(sql,engine)
    data=data.sort_values('trade_date')
    #前复权
    data['adjclose']=(data.close*data.adj_factor/data.adj_factor.iloc[-1]).values
    data['adjvol']=(data.vol*data.adj_factor/data.adj_factor.iloc[-1]).values
    data['adjopen']=(data.open*data.adj_factor/data.adj_factor.iloc[-1]).values
    data['adjhigh']=(data.high*data.adj_factor/data.adj_factor.iloc[-1]).values
    data['adjlow']=(data.low*data.adj_factor/data.adj_factor.iloc[-1]).values
    
    data=data[['trade_date','adjopen','adjhigh','adjlow','adjclose','adjvol']]
    n1=['open','high','low','close','volume']
    n2=['adjopen','adjhigh','adjlow','adjclose','adjvol']
    data.rename(columns=dict(zip(n2,n1)),inplace=True)
    data.index=pd.to_datetime(data.trade_date)
    data=data.sort_index()
    data['openinterest']=0
    data['datetime']=data.index
    data=data[['datetime','open','high','low','close','volume']]
    return data

In [31]:
import tushare as ts
def get_data2(code,date='20150101'):
    data=ts.pro_bar(ts_code=code, adj='qfq', start_date=date)
    data.index=pd.to_datetime(data.trade_date)
    data=data.sort_index()
    data['volume']=data.vol
    data['openinterest']=0
    data['datetime']=pd.to_datetime(data.trade_date)
    data=data[['datetime','open','high','low','close','volume','openinterest']]
    data=data.fillna(0)
    return data

In [40]:
def get_code_list(date='20150202'):
    #默认2010年开始回测
    dd=pro.daily_basic(trade_date=date)
    x1=dd.close<100
    #流通市值低于300亿大于50亿
    x2=dd.circ_mv>500000
    x3=dd.circ_mv<3000000
    #市盈率低于80
    x4=dd.pe_ttm<80
    #股息率大于2%
    x5=dd.dv_ttm>3
    x=x1&x2&x3&x4&x5
    stock_list=dd[x].ts_code.values
    return stock_list

In [41]:
len(get_code_list())

24

# 策略编写

In [49]:
class MyStrategy(bt.Strategy):
    # 策略参数
    params = dict(
        period=20,  # 均线周期
        look_back_days=30,
        printlog=False
    )
    
    def __init__(self):
        self.mas = dict()
        #遍历所有股票,计算20日均线
        for data in self.datas:
            self.mas[data._name] = bt.ind.SMA(data.close, period=self.p.period) 
         
    def next(self):
        #计算截面收益率
        rate_list=[]
        for data in self.datas:
            if len(data)>self.p.look_back_days:
                p0=data.close[0]
                pn=data.close[-self.p.look_back_days]
                rate=(p0-pn)/pn
                rate_list.append([data._name,rate])
                
        #股票池   
        long_list=[]
        sorted_rate=sorted(rate_list,key=lambda x:x[1],reverse=True)
        long_list=[i[0] for i in sorted_rate[:10]]
        
        # 得到当前的账户价值
        total_value = self.broker.getvalue()
        p_value = total_value*0.9/10
        for data in self.datas:
            #获取仓位
            pos = self.getposition(data).size
            if not pos and data._name in long_list and \
              self.mas[data._name][0]>data.close[0]:
                size=int(p_value/100/data.close[0])*100
                self.buy(data = data, size = size) 
                        
            if pos!=0 and data._name not in long_list or \
              self.mas[data._name][0]<data.close[0]:
                self.close(data = data)                        

    def log(self, txt, dt=None,doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')
        
    #记录交易执行情况（可省略，默认不输出结果）
    def notify_order(self, order):
        # 如果order为submitted/accepted,返回空
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 如果order为buy/sell executed,报告价格结果
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log(f'买入:\n价格:{order.executed.price:.2f},\
                成本:{order.executed.value:.2f},\
                手续费:{order.executed.comm:.2f}')
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'卖出:\n价格：{order.executed.price:.2f},\
                成本: {order.executed.value:.2f},\
                手续费{order.executed.comm:.2f}')
            
            self.bar_executed = len(self) 
            
        # 如果指令取消/交易失败, 报告结果
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失败')
        self.order = None
    
    #记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'策略收益：\n毛收益 {trade.pnl:.2f}, 净收益 {trade.pnlcomm:.2f}')

In [50]:
#加载数据
cerebro = bt.Cerebro()
for s in get_code_list():
    feed = bt.feeds.PandasData(dataname = get_data(s))
    cerebro.adddata(feed, name = s)

#回测设置
startcash=100000.0
cerebro.broker.setcash(startcash)
# 设置佣金为千分之一
cerebro.broker.setcommission(commission=0.001)
 # 添加策略
cerebro.addstrategy(MyStrategy,printlog=True) 
cerebro.run() 
#获取回测结束后的总资金
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash
#打印结果
print(f'总资金: {round(portvalue,2)}')
print(f'净收益: {round(pnl,2)}')

2015-04-27,买入:
价格:14.54,                成本:8724.53,                手续费:8.72
2015-04-27,买入:
价格:22.34,                成本:8934.14,                手续费:8.93
2015-04-28,卖出:
价格：23.06,                成本: 8934.14,                手续费9.22
2015-04-28,策略收益：
毛收益 287.97, 净收益 269.82
2015-04-29,买入:
价格:13.34,                成本:8005.45,                手续费:8.01
2015-04-29,买入:
价格:21.69,                成本:8677.39,                手续费:8.68
2015-04-30,卖出:
价格：13.73,                成本: 8005.45,                手续费8.24
2015-04-30,卖出:
价格：22.05,                成本: 8677.39,                手续费8.82
2015-04-30,策略收益：
毛收益 232.19, 净收益 215.95
2015-04-30,策略收益：
毛收益 142.25, 净收益 124.76
2015-05-06,买入:
价格:6.50,                成本:8444.97,                手续费:8.44
2015-05-07,卖出:
价格：5.97,                成本: 8444.97,                手续费7.76
2015-05-07,策略收益：
毛收益 -687.38, 净收益 -703.58
2015-05-11,卖出:
价格：12.57,                成本: 8724.53,                手续费7.54
2015-05-11,策略收益：
毛收益 -1185.52, 净收益 -1201.79
2015-05-18,买入:
价格:5.20,             

2015-09-17,买入:
价格:4.34,                成本:8681.62,                手续费:8.68
2015-09-17,卖出:
价格：6.47,                成本: 8713.02,                手续费7.77
2015-09-17,买入:
价格:12.13,                成本:8493.43,                手续费:8.49
2015-09-17,卖出:
价格：4.98,                成本: 8480.48,                手续费8.97
2015-09-17,卖出:
价格：6.65,                成本: 8321.81,                手续费8.65
2015-09-17,卖出:
价格：10.41,                成本: 8030.21,                手续费8.33
2015-09-17,策略收益：
毛收益 -943.81, 净收益 -960.30
2015-09-17,策略收益：
毛收益 491.39, 净收益 473.94
2015-09-17,策略收益：
毛收益 325.07, 净收益 308.10
2015-09-17,策略收益：
毛收益 298.42, 净收益 282.06
2015-09-18,卖出:
价格：4.20,                成本: 8681.62,                手续费8.40
2015-09-18,买入:
价格:4.83,                成本:8702.40,                手续费:8.70
2015-09-18,买入:
价格:10.29,                成本:8233.68,                手续费:8.23
2015-09-18,策略收益：
毛收益 -280.05, 净收益 -297.14
2015-09-21,买入:
价格:4.11,                成本:8625.61,                手续费:8.63
2015-09-21,卖出:
价格：4.83,                成本: 

价格:12.66,                成本:7593.26,                手续费:7.59
2016-01-22,买入:
价格:4.92,                成本:8359.17,                手续费:8.36
2016-01-22,卖出:
价格：8.87,                成本: 8736.42,                手续费7.98
2016-01-22,卖出:
价格：6.11,                成本: 8793.33,                手续费7.33
2016-01-22,策略收益：
毛收益 -756.89, 净收益 -773.60
2016-01-22,策略收益：
毛收益 -1462.96, 净收益 -1479.08
2016-01-25,买入:
价格:7.45,                成本:8196.31,                手续费:8.20
2016-01-25,卖出:
价格：4.58,                成本: 8424.43,                手续费8.24
2016-01-25,策略收益：
毛收益 -185.49, 净收益 -202.16
2016-01-26,卖出:
价格：7.39,                成本: 8196.31,                手续费8.13
2016-01-26,买入:
价格:6.36,                成本:8261.58,                手续费:8.26
2016-01-26,策略收益：
毛收益 -62.41, 净收益 -78.74
2016-01-27,卖出:
价格：11.96,                成本: 7593.26,                手续费7.18
2016-01-27,买入:
价格:4.03,                成本:8472.70,                手续费:8.47
2016-01-27,策略收益：
毛收益 -414.35, 净收益 -429.13
2016-01-29,买入:
价格:4.50,                成本:8095.66,   

价格:6.98,                成本:8381.82,                手续费:8.38
2016-05-30,卖出:
价格：7.13,                成本: 8287.52,                手续费8.55
2016-05-30,卖出:
价格：6.25,                成本: 8445.68,                手续费8.75
2016-05-30,卖出:
价格：7.69,                成本: 8516.08,                手续费8.46
2016-05-30,策略收益：
毛收益 263.60, 净收益 246.76
2016-05-30,策略收益：
毛收益 306.05, 净收益 288.85
2016-05-30,策略收益：
毛收益 -56.63, 净收益 -73.61
2016-05-31,卖出:
价格：9.22,                成本: 8060.68,                手续费8.30
2016-05-31,策略收益：
毛收益 240.39, 净收益 224.03
2016-06-01,卖出:
价格：22.58,                成本: 6569.11,                手续费6.77
2016-06-01,卖出:
价格：7.06,                成本: 8381.82,                手续费8.47
2016-06-01,买入:
价格:15.67,                成本:7833.35,                手续费:7.83
2016-06-01,策略收益：
毛收益 204.16, 净收益 190.82
2016-06-01,策略收益：
毛收益 90.02, 净收益 73.17
2016-06-02,卖出:
价格：15.68,                成本: 7833.35,                手续费7.84
2016-06-02,策略收益：
毛收益 4.35, 净收益 -11.32
2016-06-03,买入:
价格:15.70,                成本:7850.77,          

毛收益 -23.19, 净收益 -46.83
2016-10-11,卖出:
价格：8.13,                成本: 11913.79,                手续费12.19
2016-10-11,卖出:
价格：7.25,                成本: 12110.53,                手续费12.33
2016-10-11,卖出:
价格：7.58,                成本: 11966.97,                手续费12.13
2016-10-11,卖出:
价格：9.31,                成本: 11857.57,                手续费12.11
2016-10-11,策略收益：
毛收益 275.08, 净收益 250.98
2016-10-11,策略收益：
毛收益 214.49, 净收益 190.06
2016-10-11,策略收益：
毛收益 165.17, 净收益 141.07
2016-10-11,策略收益：
毛收益 248.46, 净收益 224.50
2016-10-12,买入:
价格:7.61,                成本:12177.18,                手续费:12.18
2016-10-13,买入:
价格:9.20,                成本:11960.38,                手续费:11.96
2016-10-14,买入:
价格:7.34,                成本:11749.12,                手续费:11.75
2016-10-14,卖出:
价格：5.63,                成本: 11770.65,                手续费12.40
2016-10-14,卖出:
价格：7.59,                成本: 12177.18,                手续费12.15
2016-10-14,策略收益：
毛收益 624.64, 净收益 600.48
2016-10-14,策略收益：
毛收益 -30.03, 净收益 -54.35
2016-10-17,卖出:
价格：9.22,                成本: 1

...

价格:8.77,                成本:15779.02,                手续费:15.78
2018-06-12,买入:
价格:10.00,                成本:16007.86,                手续费:16.01
2018-06-12,买入:
价格:8.97,                成本:15255.18,                手续费:15.26
2018-06-12,卖出:
价格：3.20,                成本: 15830.16,                手续费15.02
2018-06-12,卖出:
价格：8.67,                成本: 15779.02,                手续费15.60
2018-06-12,买入:
价格:12.58,                成本:15096.56,                手续费:15.10
2018-06-12,卖出:
价格：23.78,                成本: 13943.02,                手续费14.27
2018-06-12,策略收益：
毛收益 -809.50, 净收益 -840.35
2018-06-12,策略收益：
毛收益 -180.80, 净收益 -212.18
2018-06-12,策略收益：
毛收益 325.32, 净收益 297.11
2018-06-13,卖出:
价格：9.23,                成本: 15255.18,                手续费15.69
2018-06-13,策略收益：
毛收益 431.12, 净收益 400.18
2018-06-14,买入:
价格:9.00,                成本:15304.93,                手续费:15.30
2018-06-14,卖出:
价格：14.32,                成本: 15951.05,                手续费15.76
2018-06-14,策略收益：
毛收益 -194.28, 净收益 -225.98
2018-06-15,买入:
价格:64.54,           

价格：38.79,                成本: 15065.54,                手续费15.52
2018-11-02,买入:
价格:7.14,                成本:14998.39,                手续费:15.00
2018-11-02,买入:
价格:11.10,                成本:14425.48,                手续费:14.43
2018-11-02,策略收益：
毛收益 451.77, 净收益 421.19
2018-11-05,买入:
价格:39.28,                成本:11785.30,                手续费:11.79
2018-11-05,卖出:
价格：2.82,                成本: 14735.06,                手续费14.94
2018-11-05,卖出:
价格：7.18,                成本: 14998.39,                手续费15.08
2018-11-05,卖出:
价格：11.30,                成本: 14425.48,                手续费14.69
2018-11-05,卖出:
价格：4.11,                成本: 14977.55,                手续费15.22
2018-11-05,买入:
价格:22.10,                成本:13260.33,                手续费:13.26
2018-11-05,策略收益：
毛收益 209.75, 净收益 180.07
2018-11-05,策略收益：
毛收益 79.57, 净收益 49.49
2018-11-05,策略收益：
毛收益 262.28, 净收益 233.17
2018-11-05,策略收益：
毛收益 246.69, 净收益 216.49
2018-11-06,卖出:
价格：38.77,                成本: 11785.30,                手续费11.63
2018-11-06,卖出:
价格：22.05,                成

2019-05-09,买入:
价格:12.56,                成本:15069.84,                手续费:15.07
2019-05-09,买入:
价格:15.22,                成本:13698.35,                手续费:13.70
2019-05-09,卖出:
价格：3.21,                成本: 15510.03,                手续费14.13
2019-05-09,买入:
价格:6.14,                成本:14738.46,                手续费:14.74
2019-05-09,策略收益：
毛收益 -1383.33, 净收益 -1412.96
2019-05-10,买入:
价格:67.57,                成本:13513.81,                手续费:13.51
2019-05-13,卖出:
价格：69.24,                成本: 13513.81,                手续费13.85
2019-05-13,卖出:
价格：9.42,                成本: 15190.31,                手续费15.07
2019-05-13,买入:
价格:20.48,                成本:14337.49,                手续费:14.34
2019-05-13,卖出:
价格：6.15,                成本: 14738.46,                手续费14.77
2019-05-13,买入:
价格:22.52,                成本:13509.86,                手续费:13.51
2019-05-13,策略收益：
毛收益 333.92, 净收益 306.56
2019-05-13,策略收益：
毛收益 -122.63, 净收益 -152.88
2019-05-13,策略收益：
毛收益 32.43, 净收益 2.92
2019-05-14,卖出:
价格：10.02,                成本: 14629.20,        

2019-08-14,买入:
价格:19.90,                成本:15918.17,                手续费:15.92
2019-08-14,策略收益：
毛收益 306.00, 净收益 274.41
2019-08-14,策略收益：
毛收益 13.00, 净收益 -17.28
2019-08-15,买入:
价格:9.14,                成本:15538.00,                手续费:15.54
2019-08-15,卖出:
价格：80.83,                成本: 7994.36,                手续费8.08
2019-08-15,卖出:
价格：3.11,                成本: 15702.17,                手续费15.56
2019-08-15,买入:
价格:10.15,                成本:15225.00,                手续费:15.23
2019-08-15,卖出:
价格：21.38,                成本: 15918.17,                手续费17.11
2019-08-15,策略收益：
毛收益 88.39, 净收益 72.31
2019-08-15,策略收益：
毛收益 -146.29, 净收益 -177.55
2019-08-15,策略收益：
毛收益 1187.81, 净收益 1154.78
2019-08-16,卖出:
价格：4.34,                成本: 15669.99,                手续费15.62
2019-08-16,买入:
价格:6.08,                成本:15808.00,                手续费:15.81
2019-08-16,策略收益：
毛收益 -45.99, 净收益 -77.28
2019-08-20,卖出:
价格：9.35,                成本: 15538.00,                手续费15.89
2019-08-20,卖出:
价格：10.05,                成本: 15225.00,           

2020-02-10,策略收益：
毛收益 1479.00, 净收益 1449.44
2020-02-10,策略收益：
毛收益 60.00, 净收益 27.84
2020-02-11,卖出:
价格：2.66,                成本: 16043.00,                手续费16.23
2020-02-11,卖出:
价格：23.49,                成本: 16058.00,                手续费16.44
2020-02-11,卖出:
价格：4.63,                成本: 15779.00,                手续费14.35
2020-02-11,买入:
价格:8.74,                成本:15732.00,                手续费:15.73
2020-02-11,策略收益：
毛收益 183.00, 净收益 150.73
2020-02-11,策略收益：
毛收益 385.00, 净收益 352.50
2020-02-11,策略收益：
毛收益 -1426.00, 净收益 -1456.13
2020-02-12,买入:
价格:2.61,                成本:16182.00,                手续费:16.18
2020-02-12,买入:
价格:9.01,                成本:16218.00,                手续费:16.22
2020-02-12,买入:
价格:23.00,                成本:16100.00,                手续费:16.10
2020-02-13,卖出:
价格：2.61,                成本: 16182.00,                手续费16.18
2020-02-13,卖出:
价格：23.70,                成本: 16100.00,                手续费16.59
2020-02-13,策略收益：
毛收益 0.00, 净收益 -32.36
2020-02-13,策略收益：
毛收益 490.00, 净收益 457.31
2020-02-14,买入:
价格:2.54

In [51]:
cerebro = bt.Cerebro()
for s in get_code_list():
    feed = bt.feeds.PandasData(dataname = get_data(s))
    cerebro.adddata(feed, name = s)
#回测设置
startcash=100000.0
cerebro.broker.setcash(startcash)
# 设置佣金为千分之一
cerebro.broker.setcommission(commission=0.001)
# 添加策略
cerebro.addstrategy(MyStrategy) 
bt.plot_result(cerebro)

In [52]:
cerebro = bt.Cerebro()
for s in get_code_list():
    feed = bt.feeds.PandasData(dataname = get_data(s))
    cerebro.adddata(feed, name = s)
#回测设置
startcash=100000.0
cerebro.broker.setcash(startcash)
# 设置佣金为千分之一
cerebro.broker.setcommission(commission=0.001)
 # 添加策略
cerebro.addstrategy(MyStrategy) 
df00,df0,df1,df2,df3,df4=bt.out_result(cerebro)

# 使用pyecharts可视化

In [53]:
df00

,绩效指标,绩效指标值,普通交易指标,普通交易指标值,多空交易指标,多空交易指标值
0,calmar_ratio,0.087,total_trade_num,1120.0000,long_num,1114.0000
1,average_drawdown_len,52.000,total_trade_opened,1.0000,long_win_num,623.0000
2,average_drawdown_rate,2.540,total_trade_closed,1119.0000,long_lost_num,491.0000
3,average_drawdown_money,4764.860,total_trade_len,3795.0000,long_total_pnl,13329.7022
4,max_drawdown_len,371.000,long_trade_len,3785.0000,long_average_pnl,11.9656
5,max_drawdown_rate,21.800,short_trade_len,10.0000,long_win_total_pnl,213912.0270
6,max_drawdown_money,21998.060,longest_win_num,12.0000,long_win_max_pnl,4119.9337
7,average_rate,0.110,longest_lost_num,16.0000,long_lost_total_pnl,-200582.3249
8,stddev_rate,0.110,net_total_pnl,82938.6842,long_lost_max_pnl,-3757.2874
9,positive_year,6.000,net_average_pnl,74.1186,short_num,5.0000


In [54]:
from pyecharts import*
def plot_result_py(data,v,title,plot_type='line',zoom=False):
    att=data.index
    try:
        attr=att.strftime('%Y%m%d')
    except:
        attr=att
    if plot_type=='line':
        p=Line(title)
        p.add('',attr,list(data[v].round(2)),
         is_symbol_show=False,line_width=2,
        is_datazoom_show=zoom,is_splitline_show=True)
    else:
        p=Bar(title)
        p.add('',attr,[int(i*1000)/10 for i in list(data[v])],
              is_label_show=True,
        is_datazoom_show=zoom,is_splitline_show=True)
    return p

In [55]:
plot_result_py(df0,'total_value','账户价值')

In [56]:
plot_result_py(df4,'total_position_value','持仓市值')

In [57]:
plot_result_py(df3,'year_rate','年化收益%',plot_type='bar')